In [1]:
import sys
sys.path.append('../')
import os

In [2]:
from application.review_obj_class import ReviewObj
from application.namespan_class import NameSpanGenerator
import pandas as pd
%pprint

Pretty printing has been turned OFF


In [3]:
directory = "../../aps_reviews_1000/"
filenames = os.listdir(directory)
reviews = ((file.split('.')[0], open(directory + file).read()) for file in filenames)

In [4]:
review_list = [ReviewObj(file, txt) for (file, txt) in reviews]

In [5]:
ex = NameSpanGenerator.generate(review_list[20])

In [6]:
[x.collocates for x in ex.spans]

[['tribute', 'to', 'Dr._Gladden', "'s", 'zealous'], ['writers', 'of', 'Dr._Gladden', "'s", 'school'], ['probably', 'what', 'Dr._Gladden', 'means', ','], ['country', '.', 'Dr._Gladden', 'is', 'not']]

In [7]:
list(set((x for x in review_list[0].cleaned_text[:50].split() if x.isalnum())))

['Rl', 'New']

In [8]:
[x.name for x in ex.spans]

[Dr. Gladden, Dr. Gladden, Dr. Gladden, Dr. Gladden]

In [9]:
for x in review_list:
    x = NameSpanGenerator.generate(x)

In [10]:
example_reviews = [x for x in review_list if (len(x.person_names) > 3) and (len(x.pub_names) > 1)]

In [11]:
len(example_reviews)

77

In [12]:
[x.name for x in example_reviews[0].spans]

[Dr. Leopold Damrosch, Mr. Andrew Carnegie, Dr. LeopOld Damrosch, Prof. Horatio W. Parker, Mr. Parker, Lord. The, Oratorio Society, Oratorio Society, Mendelssohn Society, Symphony Society]

In [13]:
name_indices = [x.name.review_loc_toks for x in example_reviews[0].spans]

In [14]:
name_indices

[244, 423, 459, 493, 561, 731, 3, 64, 284, 433]

In [15]:
example_reviews[0].coll_toks_ind[name_indices[0]-2:name_indices[0]+3]

['z873', 'that', 'Dr._Leopold_Damrosch', 'united', 'a']

## Looking through the collocations

In [16]:
review_list[20].spans

[<application.namespan_class.NameSpan object at 0x1a35880390>, <application.namespan_class.NameSpan object at 0x1a35880150>, <application.namespan_class.NameSpan object at 0x1a358802d0>, <application.namespan_class.NameSpan object at 0x1a358803d0>]

In [17]:
[span for spans in (rev.spans for rev in review_list[:2]) for span in spans]

[<application.namespan_class.NameSpan object at 0x1a35874310>, <application.namespan_class.NameSpan object at 0x1a358742d0>, <application.namespan_class.NameSpan object at 0x1a35874390>, <application.namespan_class.NameSpan object at 0x1a35874410>, <application.namespan_class.NameSpan object at 0x1a35874550>, <application.namespan_class.NameSpan object at 0x1a35874590>, <application.namespan_class.NameSpan object at 0x1a358743d0>, <application.namespan_class.NameSpan object at 0x1a358744d0>, <application.namespan_class.NameSpan object at 0x1a35874510>, <application.namespan_class.NameSpan object at 0x1a358745d0>, <application.namespan_class.NameSpan object at 0x1a35874450>, <application.namespan_class.NameSpan object at 0x1a35874490>, <application.namespan_class.NameSpan object at 0x1a35874650>, <application.namespan_class.NameSpan object at 0x1a35874690>, <application.namespan_class.NameSpan object at 0x1a358746d0>, <application.namespan_class.NameSpan object at 0x1a35874710>, <applic

In [18]:
spans = [span for spans in (rev.spans for rev in review_list) for span in spans]

In [19]:
len(spans)

5839

In [20]:
person_spans = [span for span in spans if span.label == 'person']

### Problem situations:
    
+ Okay: 0 1 name(2) 3 4
    + 2 !< 2
+ Bad: X 0 name(1) 2 3
    + 1 < 2
    + **2 - 1 = 1**
+ Bad: X X name(0) 2 3
    + 0 < 2
    + **2 - 0 = 2**

**Pattern: span.name.review_loc_toks < 2**

+ Okay: -5 -4 name(-3) -2 -1
    + 0 1 name(2) 3 4
    + 5 toks, 5 - 3 = 2
    + 2 !> 2
+ Bad: -4 -3 name(-2) -1 X
    + 0 1 name(2) 3
    + 4 toks, 4 - 3 = 1
    + 2 > 1
    + **2 - 1 = 1**
+ Bad: -3 -2 name(-1) X X
    + 0 1 name(2)
    + 3 toks, 3 - 3 = 0
    + 2 > 0
    + **2 - 0 = 2**
+ Bad: -4 -3 name(-2) -1 X
    + 300 301 302(name) 303 X
    + 304 toks, 304 - 3 = 301
    + 302 > 301
    + **302 - 301 = 1**
+ Bad: -3 -2 name(-1) X X
    + 302 303(name) X X
    + 304 toks, 304 - 3 = 301
    + 303 > 301
    + **303 - 301 = 2**

**Pattern: span.name.review_loc_toks > (len(toks)-3)**

In [21]:
# len([span.collocates for span in person_spans if len(span.collocates) < 5])

In [22]:
# [span.collocates for span in person_spans if len(span.collocates) < 5]

In [23]:
# problem_ids = [span.review_id for span in person_spans if len(span.collocates) < 5]

In [24]:
# problem_revs = [rev for rev in review_list if rev.review_id in problem_ids]

In [25]:
# len(problem_revs)

In [26]:
# import copy

In [27]:
# for review in problem_revs:
#     for span in review.spans:
#         toks = copy.deepcopy(review.coll_toks_ind)
#         ni = span.name.review_loc_toks
#         if (ni < 2):
#             print("front:")
#             ftoks = list((2-ni)*"^")
#             ftoks.extend((toks))
#             print(ftoks[0:(ni+3+(2-ni))])
#             print()
#         if (ni > (len(toks)-3)):
#             print("back:")
#             btoks = toks
#             btoks.extend(list((ni-(len(toks)-3)) * "$"))
#             print(btoks[(ni-2):])
#             print()

In [28]:
len(person_spans)

5192

In [29]:
coll_df = pd.DataFrame()

In [30]:
coll_df['review_id'] = list(map((lambda x: x.review_id), person_spans))
coll_df['review_group'] = list(map((lambda x: x.group), person_spans))

In [31]:
person_spans[0].collocates

['Place', 'and', 'Mrs._Canfield', "'s", 'The']

In [32]:
# problem fixed
set([len(span.collocates) for span in person_spans])

{5}

In [33]:
coll_df['preceding_collocates'] = list(map((lambda x: x.collocates[:2]), person_spans))
coll_df['following_collocates'] = list(map((lambda x: x.collocates[3:]), person_spans))

coll_df['coll-2'] = list(map((lambda x: x.collocates[0]), person_spans))
coll_df['coll-1'] = list(map((lambda x: x.collocates[1]), person_spans))
coll_df['name_target'] = list(map((lambda x: x.name), person_spans))
coll_df['coll+1'] = list(map((lambda x: x.collocates[3]), person_spans))
coll_df['coll+2'] = list(map((lambda x: x.collocates[4]), person_spans))

In [37]:
coll_df['all_coll'] = coll_df.preceding_collocates + coll_df.following_collocates

In [43]:
coll_df.head()

,review_id,review_group,preceding_collocates,following_collocates,coll-2,coll-1,name_target,coll+1,coll+2,all_coll
0,137197814,21,"[Place, and]","['s, The]",Place,and,Mrs. Canfield,'s,The,"[Place, and, 's, The]"
1,137197814,22,"[it, is]","['s, own]",it,is,Mr. Anderson,'s,own,"[it, is, 's, own]"
2,137197814,1,"[Years, ago]","[,, writing]",Years,ago,Mr. George Moore,",",writing,"[Years, ago, ,, writing]"
3,137197814,26,"[Matisse, .]","[is, a]",Matisse,.,Mr. Swinnerton,is,a,"[Matisse, ., is, a]"
4,137197814,33,"[house, .]","[seems, to]",house,.,Mr. Hergesheimer,seems,to,"[house, ., seems, to]"


In [39]:
coll_df.preceding_collocates.value_counts()[:10]

[., By]         43
[,, and]        39
[,, by]         33
[,, as]         29
[of, the]       25
[,, ``]         24
[edited, by]    21
[by, the]       19
[,, that]       18
[., But]        18
Name: preceding_collocates, dtype: int64

In [40]:
coll_df.following_collocates.value_counts()[:10]

['s, book]    63
[,, of]       50
[,, who]      49
[,, and]      46
[of, the]     39
['s, ``]      39
['s, work]    36
[,, the]      33
[,, in]       32
[,, ``]       30
Name: following_collocates, dtype: int64

In [53]:
coll_df.all_coll.value_counts()[:10]

[``, says, ,, ``]        6
[., By, ,, M.]           5
[,, by, ,, of]           4
[., By, ., New]          3
[,, by, ,, is]           3
[of, ``, ,, ``]          3
[edited, by, ,, D.D.]    2
[,, by, ., (]            2
[Rev, ., ,, who]         2
[,, of, 's, ``]          2
Name: all_coll, dtype: int64

## scikit-learn mutual information?

In [93]:
ex = ["●" if "●" in x else x for x in review_list[5].coll_toks_all]

In [99]:
ex = ["^","^"] + ex + ["$","$"]

In [101]:
ex[:10]

['^', '^', 'Z*e', 'IFamiIW', 'in', 'Uomaun', 'LaW', '.', 'x', '.']

In [100]:
ex[-10:]

['as', 'a', 'whole', '.', 'ALBERT', 'W.', 'RYAN', '.', '$', '$']

In [104]:
len(ex)-5

12747

In [105]:
ex[12747]

'W.'

In [108]:
for e, tok in enumerate(ex):
    if e <= len(ex)-5:
        coll = ex[e:e+5]
        if coll[2]=="●":
            print(coll[:2] + coll[3:], 1)
        else:
            print(coll[:2] + coll[3:], 0)
    #print(coll, coll[2]=="●")

['^', '^', 'IFamiIW', 'in'] 0
['^', 'Z*e', 'in', 'Uomaun'] 0
['Z*e', 'IFamiIW', 'Uomaun', 'LaW'] 0
['IFamiIW', 'in', 'LaW', '.'] 0
['in', 'Uomaun', '.', 'x'] 0
['Uomaun', 'LaW', 'x', '.'] 0
['LaW', '.', '.', 'The'] 0
['.', 'x', 'The', 'History'] 0
['x', '.', 'History', 'and'] 0
['.', 'The', 'and', 'Prineciples'] 0
['The', 'History', 'Prineciples', 'of'] 0
['History', 'and', 'of', 'the'] 0
['and', 'Prineciples', 'the', 'Civil'] 0
['Prineciples', 'of', 'Civil', 'Law'] 0
['of', 'the', 'Law', 'of'] 0
['the', 'Civil', 'of', 'Rome'] 0
['Civil', 'Law', 'Rome', '.'] 0
['Law', 'of', '.', 'By'] 0
['of', 'Rome', 'By', 'SHELDON'] 0
['Rome', '.', 'SHELDON', 'Amos'] 0
['.', 'By', 'Amos', ','] 0
['By', 'SHELDON', ',', 'M.'] 0
['SHELDON', 'Amos', 'M.', 'A.'] 0
['Amos', ',', 'A.', 'London'] 0
[',', 'M.', 'London', '.'] 0
['M.', 'A.', '.', 'i883'] 0
['A.', 'London', 'i883', '.'] 0
['London', '.', '.', '2'] 0
['.', 'i883', '2', '.'] 0
['i883', '.', '.', 'Outlines'] 0
['.', '2', 'Outlines', 'of'] 0
['2', 

['manner', 'the', 'religious', 'conceptions'] 0
['the', 'primitive', 'conceptions', 'of'] 0
['primitive', 'religious', 'of', 'the'] 0
['religious', 'conceptions', 'the', 'Roman'] 0
['conceptions', 'of', 'Roman', 'people'] 0
['of', 'the', 'people', '.'] 0
['the', 'Roman', '.', 'The'] 0
['Roman', 'people', 'The', 'picture'] 0
['people', '.', 'picture', 'would'] 0
['.', 'The', 'would', 'of'] 0
['The', 'picture', 'of', 'necessity'] 0
['picture', 'would', 'necessity', 'be'] 0
['would', 'of', 'be', 'indistinct'] 0
['of', 'necessity', 'indistinct', 'and'] 0
['necessity', 'be', 'and', 'incomplete'] 0
['be', 'indistinct', 'incomplete', 'were'] 0
['indistinct', 'and', 'were', 'it'] 0
['and', 'incomplete', 'it', 'not'] 0
['incomplete', 'were', 'not', 'for'] 0
['were', 'it', 'for', 'the'] 0
['it', 'not', 'the', 'fact'] 0
['not', 'for', 'fact', 'that'] 0
['for', 'the', 'that', 'the'] 0
['the', 'fact', 'the', 'primitive'] 0
['fact', 'that', 'primitive', 'Roman'] 0
['that', 'the', 'Roman', 'religion'

['my', 'brother', 'Orestes', ':'] 0
['brother', ',', ':', 'make'] 0
[',', 'Orestes', 'make', 'him'] 0
['Orestes', ':', 'him', 'return'] 0
[':', 'make', 'return', 'to'] 0
['make', 'him', 'to', 'this'] 0
['him', 'return', 'this', 'country'] 0
['return', 'to', 'country', ';'] 0
['to', 'this', ';', 'hear'] 0
['this', 'country', 'hear', 'my'] 0
['country', ';', 'my', 'prayer'] 0
[';', 'hear', 'prayer', ','] 0
['hear', 'my', ',', '0'] 0
['my', 'prayer', '0', 'my'] 0
['prayer', ',', 'my', 'father'] 0
[',', '0', 'father', ';'] 0
['0', 'my', ';', 'grant'] 0
['my', 'father', 'grant', 'my'] 0
['father', ';', 'my', 'wishes'] 0
[';', 'grant', 'wishes', ';'] 0
['grant', 'my', ';', 'receive'] 0
['my', 'wishes', 'receive', 'my'] 0
['wishes', ';', 'my', 'libation.'] 0
[';', 'receive', 'libation.', '``'] 0
['receive', 'my', '``', 'And'] 0
['my', 'libation.', 'And', 'to'] 0
['libation.', '``', 'to', 'show'] 0
['``', 'And', 'show', 'that'] 0
['And', 'to', 'that', 'the'] 0
['to', 'show', 'the', 'Lares'] 0


['The', 'primitive', 'of', 'marriage'] 0
['primitive', 'form', 'marriage', 'shows'] 0
['form', 'of', 'shows', 'what'] 0
['of', 'marriage', 'what', 'an'] 0
['marriage', 'shows', 'an', 'intimate'] 0
['shows', 'what', 'intimate', 'place'] 0
['what', 'an', 'place', 'the'] 0
['an', 'intimate', 'the', 'sacred'] 0
['intimate', 'place', 'sacred', 'fire'] 0
['place', 'the', 'fire', 'held'] 0
['the', 'sacred', 'held', 'in'] 0
['sacred', 'fire', 'in', 'the'] 0
['fire', 'held', 'the', 'early'] 0
['held', 'in', 'early', 'Roman'] 0
['in', 'the', 'Roman', 'household'] 0
['the', 'early', 'household', '.'] 0
['early', 'Roman', '.', 'In'] 0
['Roman', 'household', 'In', 'later'] 0
['household', '.', 'later', 'times'] 0
['.', 'In', 'times', ','] 0
['In', 'later', ',', 'by'] 0
['later', 'times', 'by', 'a'] 0
['times', ',', 'a', 'gradual'] 0
[',', 'by', 'gradual', 'development'] 0
['by', 'a', 'development', 'of'] 0
['a', 'gradual', 'of', 'the'] 0
['gradual', 'development', 'the', 'idea'] 0
['development', '

['marriage', ',', 'without', 'this'] 0
[',', 'which', 'this', 'precedent'] 0
['which', 'without', 'precedent', 'consideration'] 0
['without', 'this', 'consideration', 'it'] 0
['this', 'precedent', 'it', 'would'] 0
['precedent', 'consideration', 'would', 'be'] 0
['consideration', 'it', 'be', 'difficult'] 0
['it', 'would', 'difficult', 'to'] 0
['would', 'be', 'to', 'understand'] 0
['be', 'difficult', 'understand', '.'] 0
['difficult', 'to', '.', 'This'] 0
['to', 'understand', 'This', 'domestic'] 0
['understand', '.', 'domestic', 'religion'] 0
['.', 'This', 'religion', 'was'] 0
['This', 'domestic', 'was', 'essentially'] 0
['domestic', 'religion', 'essentially', 'a'] 0
['religion', 'was', 'a', 'separative'] 0
['was', 'essentially', 'separative', 'influence'] 0
['essentially', 'a', 'influence', '.'] 0
['a', 'separative', '.', 'It'] 0
['separative', 'influence', 'It', 'offered'] 0
['influence', '.', 'offered', 'no'] 0
['.', 'It', 'no', 'one'] 0
['It', 'offered', 'one', 'god'] 0
['offered', '

...

['were', 'limited', 'the', 'purpose'] 0
['limited', 'by', 'purpose', 'for'] 0
['by', 'the', 'for', 'which'] 0
['the', 'purpose', 'which', 'he'] 0
['purpose', 'for', 'he', 'was'] 0
['for', 'which', 'was', 'appointed'] 0
['which', 'he', 'appointed', '.'] 0
['he', 'was', '.', 'He'] 0
['was', 'appointed', 'He', 'could'] 0
['appointed', '.', 'could', 'in'] 0
['.', 'He', 'in', 'no'] 0
['He', 'could', 'no', 'case'] 0
['could', 'in', 'case', 'transcend'] 0
['in', 'no', 'transcend', 'this'] 0
['no', 'case', 'this', 'limitation'] 0
['case', 'transcend', 'limitation', '.'] 0
['transcend', 'this', '.', 'He'] 0
['this', 'limitation', 'He', 'might'] 0
['limitation', '.', 'might', 'be'] 0
['.', 'He', 'be', 'appointed'] 0
['He', 'might', 'appointed', 'for'] 0
['might', 'be', 'for', 'some'] 0
['be', 'appointed', 'some', 'special'] 0
['appointed', 'for', 'special', 'duty'] 0
['for', 'some', 'duty', ','] 0
['some', 'special', ',', 'as'] 0
['special', 'duty', 'as', 'in'] 0
['duty', ',', 'in', 'case'] 0
['

['did', 'not', 'to', 'classes'] 0
['not', 'belong', 'classes', 'between'] 0
['belong', 'to', 'between', 'which'] 0
['to', 'classes', 'which', 'm'] 0
['classes', 'between', 'm', 'arriage'] 0
['between', 'which', 'arriage', 'was'] 0
['which', 'm', 'was', 'forbidden'] 0
['m', 'arriage', 'forbidden', '.'] 0
['arriage', 'was', '.', 'Zatrimnoniumn'] 0
['was', 'forbidden', 'Zatrimnoniumn', 'non'] 0
['forbidden', '.', 'non', 'occurred'] 0
['.', 'Zatrimnoniumn', 'occurred', 'persons'] 0
['Zatrimnoniumn', 'non', 'persons', 'married'] 0
['non', 'occurred', 'married', 'whom'] 0
['occurred', 'persons', 'whom', 'the'] 0
['persons', 'married', 'the', 'law'] 0
['married', 'whom', 'law', 'forbade'] 0
['whom', 'the', 'forbade', 'to'] 0
['the', 'law', 'to', 'marry'] 0
['law', 'forbade', 'marry', ','] 0
['forbade', 'to', ',', 'as'] 0
['to', 'marry', 'as', 'plebeians'] 0
['marry', ',', 'plebeians', 'with'] 0
[',', 'as', 'with', 'patricians'] 0
['as', 'plebeians', 'patricians', ','] 0
['plebeians', 'with', 

['.', 'Within', 'house', 'the'] 0
['Within', 'the', 'the', 'took'] 0
['the', 'house', 'took', 'place'] 0
['house', 'the', 'place', 'at'] 0
['the', 'took', 'at', 'the'] 0
['took', 'place', 'the', 'hearth'] 0
['place', 'at', 'hearth', ','] 0
['at', 'the', ',', 'in'] 0
['the', 'hearth', 'in', 'the'] 0
['hearth', ',', 'the', 'presence'] 0
[',', 'in', 'presence', 'of'] 0
['in', 'the', 'of', 'the'] 0
['the', 'presence', 'the', 'Penates'] 0
['presence', 'of', 'Penates', 'and'] 0
['of', 'the', 'and', 'all'] 0
['the', 'Penates', 'all', 'the'] 0
['Penates', 'and', 'the', 'domestic'] 0
['and', 'all', 'domestic', 'gods'] 0
['all', 'the', 'gods', '.'] 0
['the', 'domestic', '.', 'The'] 0
['domestic', 'gods', 'The', 'husband'] 0
['gods', '.', 'husband', 'and'] 0
['.', 'The', 'and', 'wife'] 0
['The', 'husband', 'wife', 'offered'] 0
['husband', 'and', 'offered', 'a'] 0
['and', 'wife', 'a', 'sacrifice'] 0
['wife', 'offered', 'sacrifice', ','] 0
['offered', 'a', ',', 'poured'] 0
['a', 'sacrifice', 'poure

['marriage', '.', 'were', ':'] 0
['.', 'They', ':', '('] 0
['They', 'were', '(', 'i'] 0
['were', ':', 'i', ')'] 0
[':', '(', ')', 'Personal'] 0
['(', 'i', 'Personal', 'capacity'] 0
['i', ')', 'capacity', ';'] 0
[')', 'Personal', ';', '('] 0
['Personal', 'capacity', '(', '2'] 0
['capacity', ';', '2', ')'] 0
[';', '(', ')', 'The'] 0
['(', '2', 'The', 'consent'] 0
['2', ')', 'consent', 'of'] 0
[')', 'The', 'of', 'all'] 0
['The', 'consent', 'all', 'persons'] 0
['consent', 'of', 'persons', 'holding'] 0
['of', 'all', 'holding', 'potestas'] 0
['all', 'persons', 'potestas', 'over'] 0
['persons', 'holding', 'over', 'the'] 0
['holding', 'potestas', 'the', 'contracting'] 0
['potestas', 'over', 'contracting', 'parties'] 0
['over', 'the', 'parties', ';'] 0
['the', 'contracting', ';', '('] 0
['contracting', 'parties', '(', '3'] 0
['parties', ';', '3', ')'] 0
[';', '(', ')', 'Absence'] 0
['(', '3', 'Absence', 'of'] 0
['3', ')', 'of', 'all'] 0
[')', 'Absence', 'all', 'restrictions'] 0
['Absence', 'of'

['the', 'wife', 'it', 'reverted'] 0
['wife', ',', 'reverted', 'to'] 0
[',', 'it', 'to', 'him'] 0
['it', 'reverted', 'him', '.'] 0
['reverted', 'to', '.', 'The'] 0
['to', 'him', 'The', ','] 0
['him', '.', ',', '-'] 0
['.', 'The', '-', 'nuptias'] 0
['The', ',', 'nuptias', 'came'] 0
[',', '-', 'came', 'to'] 0
['-', 'nuptias', 'to', 'be'] 0
['nuptias', 'came', 'be', 'considered'] 0
['came', 'to', 'considered', 'a'] 0
['to', 'be', 'a', 'necessary'] 0
['be', 'considered', 'necessary', 'correlative'] 0
['considered', 'a', 'correlative', 'to'] 0
['a', 'necessary', 'to', 'the'] 0
['necessary', 'correlative', 'the', 'dos'] 0
['correlative', 'to', 'dos', '.'] 0
['to', 'the', '.', 'The'] 0
['the', 'dos', 'The', 'wife'] 0
['dos', '.', 'wife', 'had'] 0
['.', 'The', 'had', 'complete'] 0
['The', 'wife', 'complete', 'liberty'] 0
['wife', 'had', 'liberty', 'in'] 0
['had', 'complete', 'in', 'the'] 0
['complete', 'liberty', 'the', 'management'] 0
['liberty', 'in', 'management', 'of'] 0
['in', 'the', 'of',

In [130]:
def getColl(ReviewObj):
    features = []
    targets = []
    toks = ["^","^"] + ["●" if "●" in x else x for x in ReviewObj.coll_toks_all] + ["$","$"]
    for e, tok in enumerate(toks):
        if e <= len(ex)-5:
            coll = ex[e:e+5]
            if coll[2]=="●":
                features.append(coll[:2] + coll[3:])
                targets.append(1)
            else:
                features.append(coll[:2] + coll[3:])
                targets.append(0)
    return features, targets

In [132]:
features, targets = getColl(review_list[0])

In [133]:
features = []
targets = []
for rev in review_list:
    x, y = getColl(rev)
    features.extend(x)
    targets.extend(y)

In [144]:
targets[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [139]:
for x in features[:10]:
    print(x)

['^', '^', 'IFamiIW', 'in']
['^', 'Z*e', 'in', 'Uomaun']
['Z*e', 'IFamiIW', 'Uomaun', 'LaW']
['IFamiIW', 'in', 'LaW', '.']
['in', 'Uomaun', '.', 'x']
['Uomaun', 'LaW', 'x', '.']
['LaW', '.', '.', 'The']
['.', 'x', 'The', 'History']
['x', '.', 'History', 'and']
['.', 'The', 'and', 'Prineciples']


In [136]:
feature_df = pd.DataFrame(features)

In [137]:
feature_df.head()

,0,1,2,3
0,^,^,IFamiIW,in
1,^,Z*e,in,Uomaun
2,Z*e,IFamiIW,Uomaun,LaW
3,IFamiIW,in,LaW,.
4,in,Uomaun,.,x


In [140]:
# this takes so long to load
# just a warning
feature_df = pd.get_dummies(feature_df, drop_first=True)

In [141]:
feature_df.head()

,0_',0_'',0_'s,0_(,0_),0_*us,"0_,",0_-,0_.,0_/,...,3_year,3_yearly,3_years,3_yet,3_yielding,3_you,3_young,3_your,3_z,3_●
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [146]:
len(feature_df)

1154536

In [150]:
target_df = pd.DataFrame(targets)

In [151]:
len(target_df)

1154536

In [152]:
from sklearn.feature_selection import mutual_info_classif

In [154]:
from numpy import ravel

In [155]:
targ_array = ravel(target_df)

In [156]:
targ_array[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
mi_array = mutual_info_classif(feature_df, targ_array)